<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-17 10:28:03
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.39 C
-------------------
Today PnL: 47.36 K (0.34%)
Current PnL: -12.47 L (-8.69%)
CY Booked + Current PnL: -4.39 L (-3.06%)
-------------------
Total profit:  4.26 L
Total loss:  -16.73 L
-------------------
Total Booked + Current PnL: 22.59 L (19.38%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (5.83%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 70.76 L (51.02%)
Deployed:  1.17 C
Current:  1.39 C
CAGR/XIRR %: 11.59%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-229.27,H-SC,92.71,93278.0,-2038.0,2080.0,-0.39,-2.14,2.23,0.04,162.0,-0.98,0.67,83.98,OX40N,NTT,MISC
70,SBIN,760.30,863.00,-8.66,M-LC,66.67,221805.0,20326.0,6898.0,0.64,10.09,3.11,13.51,61.0,2.95,1.60,23.96,XY25,NTT,BANKS
86,UJJIVANSFB,52.77,53.00,54.03,M-SC,50.00,133245.0,-9234.0,9860.0,-0.46,-6.48,7.40,0.44,251.0,-0.94,0.96,57.27,OX40N,NTT,BANKS
17,BRITANNIA,5190.70,6446.05,17.04,H-LC,60.42,105093.0,11660.0,10940.0,0.90,12.48,10.41,24.18,31.0,1.07,0.76,27.61,XY25,ATH,FMCG
4,APOLLOHOSP,6904.43,8285.00,-18.62,H-LC,64.58,110812.0,7246.0,13464.0,0.48,7.00,12.15,20.00,14.0,0.54,0.80,22.05,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,2347.98,4546.37,7.77,H-SC,96.88,100272.0,-691.0,95218.0,0.97,-0.68,94.96,93.63,150.0,-0.01,0.72,49.35,SR,ATH,CHEMICALS
12,BANKINDIA,116.91,190.00,-31.29,H-MC,29.69,179100.0,-708.0,113120.0,-1.33,-0.39,63.16,62.52,93.0,-0.01,1.29,33.56,XR,NTT,BANKS
61,PGHH,13388.00,18062.58,-20.87,H-MC,21.88,199500.0,-1320.0,71441.0,0.73,-0.66,35.81,34.92,92.0,-0.02,1.44,3.99,X40,ATH,FMCG
59,NATIONALUM,189.63,247.44,-50.15,H-MC,51.04,101050.0,546.0,30093.0,1.40,0.54,29.78,30.49,83.0,0.02,0.73,33.12,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,325.39,485.0,-66.94,H-MC,70.31,229556.0,-38891.0,170560.0,6.06,-14.49,74.30,49.05,82.0,-0.23,1.66,18.03,XY24,NTT,FMCG
31,EASEMYTRIP,18.12,26.4,11.35,M-SC,46.88,107240.0,-69575.0,150372.0,3.58,-39.35,140.22,45.70,197.0,-0.46,0.77,6.29,XY24,NTT,TRAVEL
75,SONACOMS,578.05,1006.0,-36.06,M-SC,17.71,82469.0,-18690.0,93578.0,3.56,-18.48,113.47,74.03,202.0,-0.20,0.60,16.29,AR,BTT,AUTO
24,COFFEEDAY,59.14,80.0,-45.67,L-SC,98.96,79430.0,-34119.0,74172.0,3.53,-30.05,93.38,35.27,268.0,-0.46,0.57,94.41,XR,NTT,HOTELS
64,RELAXO,902.64,1176.0,-27.86,H-SC,97.92,65184.0,-50354.0,85345.0,2.92,-43.58,130.93,30.28,131.0,-0.59,0.47,27.34,X40N,NTT,FOOTWEAR


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,100.00,20198.0,-29395.0,596.0,-2.87,-59.27,2.95,-58.07,201.0,-49.32,0.15,35.45,XY24,ATH,MISC
56,LTIM,5564.16,7230.20,-2.18,H-LC,9.38,188280.0,-12030.0,71998.0,-1.82,-6.01,38.24,29.94,28.0,-0.17,1.36,30.07,X200,ATH,IT
87,UNIONBANK,123.87,163.00,-4.47,M-LC,27.08,165593.0,24753.0,19739.0,-1.82,17.58,11.92,31.59,73.0,1.25,1.19,43.84,XY24,NTT,BANKS
12,BANKINDIA,116.91,190.00,-31.29,H-MC,29.69,179100.0,-708.0,113120.0,-1.33,-0.39,63.16,62.52,93.0,-0.01,1.29,33.56,XR,NTT,BANKS
72,SHALBY,261.39,327.00,903.19,M-SC,70.31,136355.0,-44788.0,90253.0,-1.17,-24.73,66.19,25.10,231.0,-0.50,0.98,9.27,XY24,NTT,HEALTHCARE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-229.27,H-SC,92.71,93278.0,-2038.0,2080.0,-0.39,-2.14,2.23,0.04,162.0,-0.98,0.67,83.98,OX40N,NTT,MISC
86,UJJIVANSFB,52.77,53.0,54.03,M-SC,50.00,133245.0,-9234.0,9860.0,-0.46,-6.48,7.40,0.44,251.0,-0.94,0.96,57.27,OX40N,NTT,BANKS
78,SYMPHONY,1306.42,1306.0,-26.52,M-SC,65.62,156597.0,-14544.0,14485.0,0.78,-8.50,9.25,-0.03,195.0,-1.00,1.13,12.55,OX40N,NTT,DURABLES
21,CERA,7989.07,8422.0,-12.17,H-SC,22.92,125532.0,-18271.0,26060.0,0.70,-12.71,20.76,5.42,155.0,-0.70,0.91,38.53,OX40N,NTT,CERAMICS
80,TATAELXSI,7332.28,7332.0,-15.80,H-MC,16.67,88368.0,-14284.0,14280.0,0.02,-13.91,16.16,-0.00,54.0,-1.00,0.64,34.33,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,93.75,78905.0,44456.0,513.0,2.26,129.05,0.65,130.54,1.0,86.66,0.57,20.66,AR,ATH,ELECTRICAL
41,HINDZINC,471.44,730.22,16.43,H-LC,1.04,190400.0,-14676.0,127244.0,0.57,-7.16,66.83,54.89,2.0,-0.12,1.37,15.21,X5K,ATH,METALS
91,VBL,492.64,672.85,-13.92,H-LC,54.17,261360.0,-3680.0,100624.0,1.19,-1.39,38.50,36.58,3.0,-0.04,1.89,11.62,X40N,ATH,FMCG
82,TCS,3794.03,4998.00,-22.56,H-LC,4.17,251987.0,-43947.0,137862.0,-0.08,-14.85,54.71,31.73,4.0,-0.32,1.82,1.20,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.98,H-LC,14.58,285541.0,23940.0,121698.0,-0.79,9.15,42.62,55.67,6.0,0.20,2.06,15.81,X40,BTT,IT


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,249.44,NaN,NaN,75.00,54540.0,13285.0,87.0,-0.16,32.20,0.16,32.41,196.0,152.70,0.39,28.40,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,21.63,H-SC,88.54,155985.0,29785.0,16394.0,0.47,23.60,10.51,36.59,153.0,1.82,1.13,39.07,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,93.75,78905.0,44456.0,513.0,2.26,129.05,0.65,130.54,1.0,86.66,0.57,20.66,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,91.10,M-MC,62.50,229577.0,4615.0,163046.0,-0.19,2.05,71.02,74.53,194.0,0.03,1.66,33.29,XY24,BTT,STEEL
53,KPIGREEN,497.21,732.01,35.49,H-SC,87.50,138600.0,13303.0,45863.0,-1.16,10.62,33.09,47.22,141.0,0.29,1.00,73.40,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,21.63,H-SC,88.54,155985.0,29785.0,16394.0,0.47,23.60,10.51,36.59,153.0,1.82,1.13,39.07,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,5.43,H-SC,38.54,118322.0,16318.0,28030.0,0.44,16.00,23.69,43.48,122.0,0.58,0.85,35.46,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,1.32,H-SC,91.15,190100.0,25430.0,43647.0,2.17,15.44,22.96,41.95,119.0,0.58,1.37,24.06,X40N,ATH,AC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,93.75,78905.0,44456.0,513.0,2.26,129.05,0.65,130.54,1.0,86.66,0.57,20.66,AR,ATH,ELECTRICAL
57,MASFIN,326.60,399.50,-9.21,H-SC,94.79,103365.0,5385.0,16487.0,-0.48,5.50,15.95,22.32,146.0,0.33,0.75,49.92,XR,ATH,FINANCE
53,KPIGREEN,497.21,732.01,35.49,H-SC,87.50,138600.0,13303.0,45863.0,-1.16,10.62,33.09,47.22,141.0,0.29,1.00,73.40,MH,ATH,POWER
94,WHIRLPOOL,1219.98,2270.00,-38.43,M-SC,55.21,103508.0,12010.0,66742.0,0.43,13.13,64.48,86.07,205.0,0.18,0.75,49.73,XR,NTT,DURABLES
45,INDIAMART,2327.09,4911.36,-46.02,H-SC,72.92,140482.0,17146.0,119817.0,-0.31,13.90,85.29,111.05,118.0,0.14,1.01,37.65,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,100.00,20198.0,-29395.0,596.0,-2.87,-59.27,2.95,-58.07,201.0,-49.32,0.15,35.45,XY24,ATH,MISC
28,DIGITIDE,188.38,249.44,NaN,NaN,75.00,54540.0,13285.0,87.0,-0.16,32.20,0.16,32.41,196.0,152.70,0.39,28.40,XY24,ATH,IT
74,SIS,477.00,477.00,2308.29,M-SC,47.92,64787.0,-16303.0,16300.0,-0.48,-20.10,25.16,0.00,237.0,-1.00,0.47,31.30,OX40N,NTT,MISC
27,DEN,52.79,75.00,135.75,L-SC,95.83,66528.0,-17936.0,53475.0,1.51,-21.24,80.38,42.07,266.0,-0.34,0.48,41.28,AR,NTT,ENTERTAINMENT
35,HATHWAY,19.74,31.40,1073.91,L-SC,85.42,67400.0,-11560.0,58200.0,0.66,-14.64,86.35,59.07,269.0,-0.20,0.49,35.23,XR,NTT,ENTERTAINMENT


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,100.00,20198.0,-29395.0,596.0,-2.87,-59.27,2.95,-58.07,201.0,-49.32,0.15,35.45,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-45.67,L-SC,98.96,79430.0,-34119.0,74172.0,3.53,-30.05,93.38,35.27,268.0,-0.46,0.57,94.41,XR,NTT,HOTELS
64,RELAXO,902.64,1176.00,-27.86,H-SC,97.92,65184.0,-50354.0,85345.0,2.92,-43.58,130.93,30.28,131.0,-0.59,0.47,27.34,X40N,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,7.77,H-SC,96.88,100272.0,-691.0,95218.0,0.97,-0.68,94.96,93.63,150.0,-0.01,0.72,49.35,SR,ATH,CHEMICALS
27,DEN,52.79,75.00,135.75,L-SC,95.83,66528.0,-17936.0,53475.0,1.51,-21.24,80.38,42.07,266.0,-0.34,0.48,41.28,AR,NTT,ENTERTAINMENT


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.37
1,25,40.73
2,50,69.82


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.16
LC    34.76
MC    22.00
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.78
X40N     14.34
X40      12.47
XY25     10.93
XR        9.39
OX40N     8.56
AR        8.12
X5K       2.15
MH        1.73
X200      1.36
SR        1.20
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.98
H-SC    25.68
H-MC    18.94
M-SC    14.94
M-LC     5.75
M-MC     2.70
L-SC     1.54
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.13,0.07,33.38
IT,11.82,-11.03,65.76
FINANCE,8.58,-9.90,49.77
BANKS,7.74,-6.17,49.49
MISC,5.97,-16.06,70.65
PAINTS,5.53,-14.06,36.55
ELECTRICAL,5.27,-0.82,48.60
HEALTHCARE,4.36,0.00,26.55
AUTO,4.31,-13.18,62.95


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2584899.0,25
AR,1026523.0,10
XR,993341.0,13
X40,728049.0,10
X40N,564966.0,12
XY25,424324.0,8
OX40N,288274.0,11
SR,176840.0,2
X5K,141195.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2268151.0,26
M-SC,1561233.0,20
H-LC,1308507.0,21
H-MC,1085123.0,15
M-MC,303566.0,2
M-LC,263645.0,4
L-SC,185847.0,3
L-MC,52767.0,1
L-LC,46330.0,1


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,883243.0,7
M-SC,XY24,768532.0,7
H-SC,AR,515881.0,3
H-LC,X40,469164.0,6
H-MC,XY24,422758.0,4
H-SC,XR,326259.0,4
M-MC,XY24,303566.0,2
H-LC,X40N,292851.0,6
M-SC,XR,270496.0,4


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
